In [1]:
import sys
sys.path.append('../')
from Code.envs.SequentialMNIST import SequentialMNIST
from Code.train import train, OptWrapper
from Code.everything5 import build_standard_loop
import torch
import torch.nn as nn
from PIL import Image
import numpy as np

In [2]:
MAIN_DECAY = np.exp(-1/(700)*0.5)
ADAP_DECAY = np.exp(-1/(700*2))

spec = {
    'control_config': {
        'neuron_type': 'LIF',
        'n_neurons': 100,
        'BETA': 0.8,
        '1-beta': 'improved',
        'SPIKE_FN': 'bellec'
    },
    'mem_config': {
        'neuron_type': 'FlipFlop',
        'n_neurons': 20,
        'BETA': 0.8,
        '1-beta': 'improved',
        'SPIKE_FN': 'bellec',
        'ADAPSCALE': 30,
        'ADAPDECAY': None, #TODO: set this
        'OFFSET': None,
        'DECAY': MAIN_DECAY
    },
    'lr': 0.001,
    'lr_decay': 0.9,
    'iterations': 5000,
    'batch_size': 64,
    'architecture': '1L'
}

In [3]:
DEVICE = torch.device('cuda')


train_problem = SequentialMNIST(spec['iterations'], spec['batch_size'], DEVICE, '../')
val_problem = SequentialMNIST(-1, spec['batch_size'], DEVICE, '../', validate=True)


n_in, n_out, input_rate = train_problem.get_infos()


In [4]:
from Code.everything5 import OuterWrapper, DynNetwork, ParallelNetwork2, SequenceWrapper, MeanModule, BaseNeuron

loop = build_standard_loop(spec, n_in, input_rate)
out_neuron_size = spec['control_config']['n_neurons'] + spec['mem_config']['n_neurons']

outer = {
    'input': n_in,
    'loop': [['input'], SequenceWrapper(ParallelNetwork2(loop)), None],
    'mean': [['loop'], MeanModule(out_neuron_size, -56), None],
    'output': [['mean'], BaseNeuron(n_out, None), nn.Linear]
}

model = OuterWrapper(DynNetwork(outer))
model.to(DEVICE)

{'control': [[('input', 1), ('control', 1), ('mem', 1)], LIFNeuron(), <class 'torch.nn.modules.linear.Linear'>], 'mem': [[('input', 1), ('control', 1), ('mem', 1)], NewFlipFlop(
  (lif_on): LIFNeuron()
  (lif_off): LIFNeuron()
), <class 'torch.nn.modules.linear.Linear'>], 'output': [[('control', 1), ('mem', 1)], BaseNeuron(), None]}


OuterWrapper(
  (model): DynNetwork(
    (layers): ModuleDict(
      (loop): SequenceWrapper(
        (model): ParallelNetwork2(
          (layers): ModuleDict(
            (control): LIFNeuron()
            (control_synapse): Linear(in_features=201, out_features=100, bias=True)
            (mem): NewFlipFlop(
              (lif_on): LIFNeuron()
              (lif_off): LIFNeuron()
            )
            (mem_synapse): Linear(in_features=201, out_features=20, bias=True)
            (output): BaseNeuron()
          )
        )
      )
      (mean): MeanModule()
      (output_synapse): Linear(in_features=120, out_features=10, bias=True)
      (output): BaseNeuron()
    )
  )
)

In [5]:
optimizer = OptWrapper(model.parameters(), spec['lr'], spec['lr_decay'], 2500)

In [6]:
train(train_problem, val_problem, optimizer, model, None)

KeyboardInterrupt: 